# Bollinger Band Strategy – Jupyter Backtest Version

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from kiteconnect import KiteConnect

# === SETUP ===
API_KEY = "your_api_key"
API_SECRET = "your_api_secret"
ACCESS_TOKEN = "your_access_token"
SYMBOL = "RELIANCE"
INTERVAL = "5minute"

kite = KiteConnect(api_key=API_KEY)
kite.set_access_token(ACCESS_TOKEN)

# === FETCH HISTORICAL DATA ===
from_date = "2024-06-01"
to_date = "2024-06-25"
data = kite.historical_data(
    instrument_token=kite.ltp('NSE:' + SYMBOL)[f'NSE:{SYMBOL}']['instrument_token'],
    from_date=from_date,
    to_date=to_date,
    interval=INTERVAL
)
df = pd.DataFrame(data)

# === BOLLINGER CALCULATION ===
def calculate_bollinger(df, period=20, stddev=2.5):
    df['median'] = df['close'].rolling(window=period).mean()
    df['std'] = df['close'].rolling(window=period).std()
    df['upper_band'] = df['median'] + stddev * df['std']
    df['lower_band'] = df['median'] - stddev * df['std']
    return df

df = calculate_bollinger(df)

# === STRATEGY SIMULATION ===
position = None
entry_price = 0
sl = 0
trades = []

for i in range(21, len(df)):
    row = df.iloc[i]
    prev = df.iloc[i - 1]

    if position is None:
        if prev['close'] < prev['upper_band'] and row['close'] > row['upper_band'] * 1.005:
            position = 'SELL'
            entry_price = row['close']
            sl = entry_price * 1.01
            trades.append((row['date'], 'SELL', row['close']))

        elif prev['close'] > prev['lower_band'] and row['close'] < row['lower_band'] * 0.995:
            position = 'BUY'
            entry_price = row['close']
            sl = entry_price * 0.99
            trades.append((row['date'], 'BUY', row['close']))

    elif position == 'SELL':
        if row['close'] >= sl:
            trades.append((row['date'], 'EXIT SELL SL', row['close']))
            position = None
        elif row['close'] <= row['median']:
            trades.append((row['date'], 'PARTIAL BUY (SELL to median)', row['close']))
            position = None

    elif position == 'BUY':
        if row['close'] <= sl:
            trades.append((row['date'], 'EXIT BUY SL', row['close']))
            position = None
        elif row['close'] >= row['median']:
            trades.append((row['date'], 'PARTIAL SELL (BUY to median)', row['close']))
            position = None

# === TRADE LOG ===
trades_df = pd.DataFrame(trades, columns=['Date', 'Action', 'Price'])
print(trades_df)

# === PLOT ===
plt.figure(figsize=(16, 6))
plt.plot(df['date'], df['close'], label='Close Price', color='black')
plt.plot(df['date'], df['upper_band'], label='Upper Band', linestyle='--', color='green')
plt.plot(df['date'], df['median'], label='Median', linestyle='--', color='blue')
plt.plot(df['date'], df['lower_band'], label='Lower Band', linestyle='--', color='red')

for index, row in trades_df.iterrows():
    if 'SELL' in row['Action']:
        plt.scatter(row['Date'], row['Price'], color='red', marker='v')
    elif 'BUY' in row['Action']:
        plt.scatter(row['Date'], row['Price'], color='green', marker='^')

plt.title(f"Bollinger Band Strategy Backtest – {SYMBOL}")
plt.legend()
plt.grid()
plt.show()